In [ ]:
#Step 1: Load and Prepare Your Data
#Assume your dataset is a CSV with the following relevant columns:
#Address
#MAK
#Latitude
#Longitude
#Type (Residential/Business)


import pandas as pd

import pandas as pd


# Load your cleaned Melissa dataset
df = pd.read_csv("/content/Melissa_AddressCheck_LatitudeLongitude.csv")

# Preview
print(df.head())



                         Address Address Type        Lat        Long
0  515 E Peltason Dr, Irvine, CA  Residential  33.642963 -117.836463
1        4 Alcott Ct, Irvine, CA  Residential  33.640275 -117.841219
2        16 Joyce Ct, Irvine, CA  Residential  33.634868 -117.841251
3     11 Murasaki St, Irvine, CA  Residential  33.640608 -117.835352
4     10 Whistler Ct, Irvine, CA  Residential  33.635299 -117.840373


🗺️ What’s happening?
You’re figuring out how far apart each address is from every other address.
This uses the haversine formula, which calculates the shortest path between two points on the Earth (like drawing a line between them on a globe).

🔍 Why?
To find the best route, you need to know the distances between all pairs of locations.

In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    φ1 = math.radians(lat1)
    φ2 = math.radians(lat2)
    Δφ = math.radians(lat2 - lat1)
    Δλ = math.radians(lon2 - lon1)

    a = math.sin(Δφ / 2)**2 + math.cos(φ1) * math.cos(φ2) * math.sin(Δλ / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c


 ## Building a Network Graph

 What’s happening?
You create a graph where:

Each node is an address

Each edge (connection) shows how far one address is from another

🔍 Why?
This gives you a visual and logical structure the computer can use to find the shortest route — like connecting the dots between delivery stops.

In [ ]:
import networkx as nx

G = nx.Graph()

# Add nodes
for idx, row in df.iterrows():
    G.add_node(idx, pos=(row['Lat'], row['Long']))

# Add weighted edges (distances)
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        dist = haversine(df.loc[i, 'Lat'], df.loc[i, 'Long'],
                         df.loc[j, 'Lat'], df.loc[j, 'Long'])
        G.add_edge(i, j, weight=dist)


## Traveling Salesman Problem
What’s happening?
You ask Python to find the shortest possible route that starts at UCI, visits every address once, and comes back to UCI.

🔍 Why?
This gives you the most efficient path to visit all locations without wasting time or distance — like a pizza driver figuring out the quickest loop to deliver to all houses.

In [ ]:
from networkx.algorithms.approximation import traveling_salesman_problem

# Solve TSP starting at node 0 (UCI)
tsp_path = traveling_salesman_problem(G, cycle=True, weight='weight')

# Calculate total distance
# Print step-by-step distances and calculate total
print("Step-by-step route and distances:")
total_distance = 0

for i in range(len(tsp_path) - 1):
    node1, node2 = tsp_path[i], tsp_path[i + 1]
    name1 = df.loc[node1, 'Address']
    name2 = df.loc[node2, 'Address']
    coord1 = df.loc[node1, ['Lat', 'Long']]
    coord2 = df.loc[node2, ['Lat', 'Long']]

    dist = haversine(coord1[0], coord1[1], coord2[0], coord2[1])
    total_distance += dist

    print(f"{name1} ➝ {name2} = {dist:.2f} km")

print(f"\n🧭 Total route distance: {total_distance:.2f} km")



# Print path
print("Optimal path (approx):", tsp_path)


In [ ]:
print(df.columns)


Index(['Address', 'Address Type', 'Lat', 'Long'], dtype='object')


In [13]:
import pandas as pd
import numpy as np
import folium
import math
import networkx as nx
from networkx.algorithms.approximation import traveling_salesman_problem

def interpolate_hex_color(val, min_val, max_val, start_hex="#0000FF", end_hex="#FF0000"):
    # If income value is missing or invalid, return a default gray
    if pd.isna(val) or pd.isna(min_val) or pd.isna(max_val) or (max_val == min_val):
        return "#808080"  # neutral gray

    # Normalize the value
    ratio = (val - min_val) / (max_val - min_val)

    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip("#")
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    def rgb_to_hex(rgb):
        return "#{:02x}{:02x}{:02x}".format(*rgb)

    start_rgb = hex_to_rgb(start_hex)
    end_rgb = hex_to_rgb(end_hex)

    interp_rgb = tuple(int(start + (end - start) * ratio) for start, end in zip(start_rgb, end_rgb))
    return rgb_to_hex(interp_rgb)


# Load your dataset
df = pd.read_csv("/content/Melissa_Final_Merge.csv")

# Clean column names
df.columns = df.columns.str.strip().str.lower()

# Clean and convert 'household income' to numeric (using lower bound)
df['household income'] = df['household income'].astype(str)  # Ensure it's string
df['household income'] = df['household income'].str.extract(r'(\d{2,3},?\d{3})')[0]  # Get lower bound
df['household income'] = df['household income'].str.replace(',', '', regex=True)
df['household income'] = pd.to_numeric(df['household income'], errors='coerce')

# Check the range
min_income = df['household income'].min()
max_income = df['household income'].max()
print("Min income:", min_income)
print("Max income:", max_income)


# Flag households with children
df['has_children'] = df['children age range'].notna()

# Drop rows without coordinates
df = df.dropna(subset=['lat', 'long'])

# Haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    Δφ = math.radians(lat2 - lat1)
    Δλ = math.radians(lon2 - lon1)
    a = math.sin(Δφ / 2)**2 + math.cos(φ1) * math.cos(φ2) * math.sin(Δλ / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Build graph
G = nx.Graph()
for idx, row in df.iterrows():
    G.add_node(idx, pos=(row['lat'], row['long']))

for i in range(len(df)):
    for j in range(i + 1, len(df)):
        dist = haversine(df.loc[i, 'lat'], df.loc[i, 'long'], df.loc[j, 'lat'], df.loc[j, 'long'])
        G.add_edge(i, j, weight=dist)

# Solve TSP
tsp_path = traveling_salesman_problem(G, cycle=True, weight='weight')

# Create Folium map
m = folium.Map(location=[df.loc[0, 'lat'], df.loc[0, 'long']], zoom_start=13)
total_distance = 0

# Draw route lines
for i in range(len(tsp_path) - 1):
    idx1, idx2 = tsp_path[i], tsp_path[i + 1]
    lat1, lon1 = df.loc[idx1, ['lat', 'long']]
    lat2, lon2 = df.loc[idx2, ['lat', 'long']]
    dist = haversine(lat1, lon1, lat2, lon2)
    total_distance += dist
    folium.PolyLine(
        locations=[(lat1, lon1), (lat2, lon2)],
        color='blue', weight=2,
        popup=f"{df.loc[idx1, 'address']} ➝ {df.loc[idx2, 'address']}<br>Distance: {dist:.2f} km"
    ).add_to(m)

# Add markers with income coloring and info
income_threshold = 75000
for order, idx in enumerate(tsp_path[:-1]):
    row = df.loc[idx]
    income = row['household income']

    # Get color based on income
    color = interpolate_hex_color(income, min_income, max_income, "#0000FF", "#FF0000")  # blue → red

    # Use CircleMarker for better control
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        popup=f"{order + 1}. {row['address']}<br>Income: ${income:,}",
        tooltip=row['mak']
    ).add_to(m)

    # Optional: order number label on top
    folium.map.Marker(
        [row['lat'], row['long']],
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; font-weight: bold; color: black;">{order + 1}</div>'
        )
    ).add_to(m)



# Start marker showing total route distance
folium.Marker(
    location=[df.loc[tsp_path[0], 'lat'], df.loc[tsp_path[0], 'long']],
    popup=f"<b>Start:</b> {df.loc[tsp_path[0], 'address']}<br><b>Total Distance:</b> {total_distance:.2f} km",
    icon=folium.Icon(color='darkblue', icon='info-sign')
).add_to(m)

# Save the map
m.save("melissa_final_map.html")
print("✅ Map saved as 'melissa_final_map.html'")

m


Min income: 15000.0
Max income: 150001.0
✅ Map saved as 'melissa_final_map.html'


In [11]:
print("Min income:", min_income)
print("Max income:", max_income)

Min income: nan
Max income: nan
